<a href="https://www.kaggle.com/code/zlatanshev/titanic-df-shevchenko?scriptVersionId=199100069" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# Standard python libraries
import time

# Essential DS libraries
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

import matplotlib.pyplot as plt
import seaborn as sns

import plotly
import plotly.express as px

import category_encoders as ce

from sklearn import preprocessing

import geopy.geocoders
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

from sklearn import tree 
from sklearn import ensemble 

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import RidgeCV

from sklearn.feature_selection import RFE

import optuna

from sklearn.model_selection import cross_val_score

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
test_data.info()

In [ ]:
# для корректной обработки признаков объединяем трейн и тест в один датасет
train_data['sample'] = 1 # помечаем где у нас трейн
test_data['sample'] = 0 # помечаем где у нас тест
test_data['Survived'] = 0 # в тесте у нас нет значения Survived, мы его должны предсказать, по этому пока просто заполняем нулями

titan_data = pd.concat([train_data, test_data], ignore_index=True).reset_index(drop=True)

In [ ]:
titan_data.head(5)

In [ ]:
titan_data.Name[:20]

In [ ]:
titan_data.info()

In [ ]:
low_information_cols = [] 

#цикл по всем столбцам
for col in titan_data.columns:
    #наибольшая относительная частота в признаке
    top_freq = titan_data[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = titan_data[col].nunique() / titan_data[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.7:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.7:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
# По общим правилам при более 70 % уникальных значений в признаки его следует удалить, но, считаю, что признак важен для дальнейшей модели


In [ ]:
titan_data.isnull().sum()

In [ ]:
titan_data.Cabin = titan_data.Cabin.fillna(0).apply(lambda x: x if x == 0 else 1)


In [ ]:
titan_data.Fare = titan_data.Fare.fillna(titan_data.Fare.median())

In [ ]:
titan_data.Embarked = titan_data.Embarked.fillna(titan_data.Embarked.mode()[0])

In [ ]:
titan_data.Age = pd.cut(titan_data.Age, [0, 10, 20, 30, 40 , 50, 60, 9999], labels = ['<10', '10-20', '20-30', '30-40', '40-50', '50-60', '+60']).astype('object')

In [ ]:
titan_data.head()

In [ ]:
titan_data.info()

In [ ]:
titan_data.describe()

In [ ]:
titan_data.describe(include='object')

In [ ]:
df_age_pred = titan_data.copy()
 # В качестве тестовой выборки возьмем строки с пропусками в признаке Age
test_df = df_age_pred[df_age_pred['Age'].isnull()]
# И удалим эти строчки из таблицы
df_age_pred.dropna(inplace=True)
 # Определим целевой признак и факторы
y_train = df_age_pred['Age']
X_train = df_age_pred.drop(['Name','Age','Ticket', 'PassengerId', 'sample'], axis=1)
X_test = test_df.drop(['Name','Age','Ticket', 'PassengerId', 'sample'], axis=1)

In [ ]:
X_train_dummies = pd.get_dummies(X_train)
X_test_dummies = pd.get_dummies(X_test)

In [ ]:
model = LogisticRegression(
    multi_class='multinomial', #мультиклассовая классификация
    max_iter=1000, #количество итераций, выделенных на сходимость
    random_state=42 #генерация случайных чисел
)
model.fit(X_train_dummies, y_train)
 
y_pred = model.predict(X_train_dummies)
y_pred.shape

In [ ]:
for i, ni in enumerate(test_df.index):
    titan_data.loc[ni, 'Age'] = y_pred[i]

In [ ]:
titan_data.info()

In [ ]:
plt.figure()
sns.pairplot(titan_data, hue= 'Survived')
plt.show()

In [ ]:
titan_data.head(5)

In [ ]:
titan_data['Relatives'] = titan_data['SibSp'] + titan_data['Parch']

In [ ]:
titan_data.groupby('Ticket', as_index = False)['PassengerId'].count().sort_values(by='PassengerId')

In [ ]:
titan_data['Sex'] = titan_data['Sex'].apply(lambda x: 1 if x == 'male' else 0)

In [ ]:
# преобразуйте уровни образования
label_encoder = LabelEncoder()
titan_data['Pclass'] = label_encoder.fit_transform(titan_data['Pclass'])
titan_data['Age'] = label_encoder.fit_transform(titan_data['Age'])
titan_data['SibSp'] = label_encoder.fit_transform(titan_data['SibSp'])
titan_data['Parch'] = label_encoder.fit_transform(titan_data['Parch'])
titan_data['Relatives'] = label_encoder.fit_transform(titan_data['Relatives'])

In [ ]:
titan_data = titan_data.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [ ]:
visual_features = list(titan_data.columns)[2:11]
visual_features

In [ ]:
#Создаём цикл для всех признаков из списка 
n = len(visual_features) #число признаков
fig, axes = plt.subplots(n, 2, figsize=(20, 40)) #фигура+n*2 координатных плоскостей
for i, feature in enumerate(visual_features):
    #Строим пирог для процентного соотношения каждой из категорий в данных
    count_data = titan_data[feature].value_counts().rename(feature)
    axes[i][0].pie(count_data, autopct='%.1f%%', labels=count_data.index)
    axes[i][0].set_title(feature)
    sns.countplot(data=titan_data, x=feature, hue='Survived', ax=axes[i][1])
    axes[i][1].set_title(feature)
plt.tight_layout() #выравнивание графиков

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
hisplot = sns.histplot(titan_data, x='Fare', ax=axes[0]);
hisplot.set_title('Гистограмма Fare');
boxplot = sns.boxplot(titan_data, x='Fare', ax=axes[1]);
boxplot.set_title('Коробчатая диаграмма Fare');

In [ ]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    print(round(lower_bound))
    upper_bound = mu + 3 * sigma
    print(round(upper_bound))
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
# Применим эту функцию к таблице sber_data и признаку mkad_km, а также выведем размерности результатов:

outliers, cleaned = outliers_z_score(titan_data, 'Fare', log_scale=False)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

In [ ]:
# напишим функцию для отлавливания выбросов по методу Тьюки
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    print(round(lower_bound))
    upper_bound = quartile_3 + (iqr * 1.5)
    print(round(upper_bound))
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_iqr(titan_data, 'Fare')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

In [ ]:
new_test_data = titan_data.query('sample == 0').drop(['sample'], axis=1)

In [ ]:
titan_data = titan_data[(titan_data['Fare'] >= -122) & (titan_data['Fare'] <= 189)]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
hisplot = sns.histplot(titan_data, x='Fare', ax=axes[0]);
hisplot.set_title('Гистограмма Fare');
boxplot = sns.boxplot(titan_data, x='Fare', ax=axes[1]);
boxplot.set_title('Коробчатая диаграмма Fare');

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
hisplot = sns.histplot(titan_data, x='PassengerId', ax=axes[0]);
hisplot.set_title('Гистограмма PassengerId');
boxplot = sns.boxplot(titan_data, x='PassengerId', ax=axes[1]);
boxplot.set_title('Коробчатая диаграмма PassengerId');

In [ ]:
survived_values = titan_data.Survived.value_counts(normalize=True).__round__(2)
survived_values

In [ ]:
ax = sns.countplot(titan_data, x= 'Survived');
ax.bar_label(ax.containers[0], fontsize=10);
ax.set_title('Сбалансированность survived');

In [ ]:
titan_data.head(3)

In [ ]:
df_columns = ['PassengerId', 'Pclass', 'Sex', 'Age', 'Parch', 'Fare',
       'Relatives', 'sample', 'Survived']

In [ ]:
df_dummies = titan_data[df_columns]

In [ ]:
df_columns_test = ['PassengerId', 'Pclass', 'Sex', 'Age', 'Parch', 'Fare',
       'Relatives', 'Survived']

In [ ]:
new_test_data

In [ ]:
new_test_data = new_test_data[df_columns_test]

In [ ]:
# df_dummies = pd.get_dummies(titan_data, columns=['Embarked'], dtype=int)

In [ ]:
# new_test_data = pd.get_dummies(new_test_data, columns=['Embarked'], dtype=int)

In [ ]:
df_dummies.head()

In [ ]:
fig, (ax1) = plt.subplots(ncols=1, figsize=(18, 15))
sns.heatmap(df_dummies.corr(), annot=True, ax=ax1)

In [ ]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

corr_pairs = get_top_abs_correlations(df_dummies, 30)
corr_pairs

In [ ]:
# Теперь выделим тестовую часть
train_data = df_dummies.query('sample == 1').drop(['sample'], axis=1)


In [ ]:
train_data

In [ ]:
y = train_data.Survived            # наш таргет
X = train_data.drop(['Survived'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = RANDOM_SEED, test_size = 0.2)

In [ ]:
X_train

In [ ]:
new_test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
new_test_data = new_test_data.drop('Survived', axis=1)

In [ ]:
new_test_data

In [ ]:
test_scaler = preprocessing.MinMaxScaler()
test_scaler.fit(new_test_data)
test_data_scaled = test_scaler.transform(new_test_data)

In [ ]:
test_data_scaled

In [ ]:
# обучите логистическую регрессию и рассчитайте метрики качества
linear_model = LogisticRegression(solver='sag',random_state=42, max_iter=1000)
linear_model.fit(X_train_scaled, y_train)

In [ ]:
#Делаем предсказание для тестовой выборки
y_train_pred = linear_model.predict(X_train_scaled)
y_test_pred = linear_model.predict(X_test_scaled)
#Вывод отчет о метриках классификации
print(metrics.classification_report(y_train, y_train_pred))
print(metrics.classification_report(y_test, y_test_pred))

In [ ]:
# подберите оптимальные параметры с помощью gridsearch
param_grid = {'min_samples_split': [2, 3, 5, 7, 10],
              'min_samples_leaf': [2, 3, 5, 7, 10],
              'max_depth':[2,3,5,7,9,11],
              'criterion':['gini', 'entropy']
              }
            
grid_search_tree = GridSearchCV(
    estimator=tree.DecisionTreeClassifier(random_state=42), 
    param_grid=param_grid,
    scoring='f1', 
    cv=2, 
    n_jobs = -1
)  
%time grid_search_tree.fit(X_train_scaled, y_train) 
y_train_pred = grid_search_tree.predict(X_train_scaled)
print(metrics.classification_report(y_train, y_train_pred))
y_test_pred = grid_search_tree.predict(X_test_scaled)
print(metrics.classification_report(y_test, y_test_pred))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_tree.best_params_))

In [ ]:
# обучим на наших данных случайный лес
#Создаём объект класса случайный лес
rf_df_bank = ensemble.RandomForestClassifier(
    n_estimators=100, #число деревьев
    criterion='gini', #критерий эффективности
    max_depth=7, #максимальная глубина дерева
    min_samples_leaf=5, #число признаков из метода случайных подространств
    min_samples_split=2,
    random_state=42 #генератор случайных чисел
)
#Обучаем модель 
rf_df_bank.fit(X_train_scaled, y_train)

#Делаем предсказание класса
y_train_pred = rf_df_bank.predict(X_train_scaled)
print(metrics.classification_report(y_train, y_train_pred))
y_test_pred = rf_df_bank.predict(X_test_scaled)
#Выводим отчёт о метриках
print(metrics.classification_report(y_test, y_test_pred))

In [ ]:
#Нас интересует только вероятность класса (второй столбец)
y_test_proba_only_rf = rf_df_bank.predict_proba(X_test_scaled)[:, 1]
#Для удобства завернем numpy-массив в pandas Series
y_test_proba_only_rf = pd.Series(y_test_proba_only_rf)
#Создадим списки, в которых будем хранить значения метрик 
recall_scores = []
precision_scores = []
f1_scores = []
#Сгенерируем набор вероятностных порогов в диапазоне от 0.1 до 1
thresholds = np.arange(0.1, 1, 0.05)
#В цикле будем перебирать сгенерированные пороги
for threshold in thresholds:
    #В противном случае - к классу 0
    y_test_pred = y_test_proba_only_rf.apply(lambda x: 1 if x>threshold else 0)
    #Считаем метрики и добавляем их в списки
    recall_scores.append(metrics.recall_score(y_test, y_test_pred))
    precision_scores.append(metrics.precision_score(y_test, y_test_pred))
    f1_scores.append(metrics.f1_score(y_test, y_test_pred))

#Визуализируем метрики при различных threshold
fig, ax = plt.subplots(figsize=(10, 4)) #фигура + координатная плоскость
#Строим линейный график зависимости recall от threshold
ax.plot(thresholds, recall_scores, label='Recall')
#Строим линейный график зависимости precision от threshold
ax.plot(thresholds, precision_scores, label='Precision')

#Строим линейный график зависимости F1 от threshold
ax.plot(thresholds, f1_scores, label='F1-score')
#Даем графику название и подписи осям
ax.set_title('Recall/Precision dependence on the threshold')
ax.set_xlabel('Probability threshold')
ax.set_ylabel('Score')
ax.legend();

In [ ]:

#Задаем оптимальный порог вероятностей
threshold_opt = 0.42
y_test_pred_only_rf = y_test_proba_only_rf.apply(lambda x: 1 if x > threshold_opt else 0)
#Считаем метрики

print(metrics.classification_report(y_test, y_test_pred_only_rf))

In [ ]:
# Используем для классификации градиентный бустинг и сравним качество со случайным лесом
# Создаем модель градиентного бустинга
gb = GradientBoostingClassifier(
    loss='log_loss', #функция потерь
    learning_rate=0.015, #темп обучения
    n_estimators=300, #число деревьев
    max_depth=5,
    min_samples_leaf=5,#максимальная глубина дерева
    random_state=42 #генератор случайных чисел
)
# Обучаем модель
gb.fit(X_train_scaled, y_train)
# Формируем предсказание для тестовой выборки
y_pred_gb = gb.predict(X_test_scaled)
# Посмотрим на основные метрики классификации
print(classification_report(y_test, y_pred_gb))

In [ ]:
# Объединим уже известные алгоритмы с помощью стекинга 
# Создаем список кортежей вида: (наименование модели, модель)
estimators = [
    ('gb', GradientBoostingClassifier(
    loss='log_loss', #функция потерь
    learning_rate=0.015, #темп обучения
    n_estimators=300, #число деревьев
    max_depth=5,
    min_samples_leaf=5,#максимальная глубина дерева
    random_state=42 #генератор случайных чисел
    )),
    ('lr', LogisticRegression(
    solver='sag',
    random_state=42, 
    max_iter=1000
    )),
    ('dt', tree.DecisionTreeClassifier(
    criterion='entropy',
    max_depth= 7,
    min_samples_leaf= 2, 
    min_samples_split= 10,
    random_state=42
    ))
]

# Создаем объект класса стекинг
reg = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()
)
# Обучаем модель
reg.fit(X_train_scaled, y_train)
# Формируем предсказание для тестовой выборки
y_pred_stack = reg.predict(X_test_scaled)
# Оцениваем качество по метрике
print(metrics.classification_report(y_test, y_pred_stack))

In [ ]:
X_train_scaled = pd.DataFrame(data=X_train_scaled, columns=X_train.columns)
estimator = GradientBoostingClassifier()
selector = RFE(estimator, n_features_to_select=7, step=1)
selector = selector.fit(X_train_scaled, y_train)
 
selector.get_feature_names_out()

In [ ]:
estimator = gb
selector = RFE(estimator, n_features_to_select=6, step=1)
selector = selector.fit(X_train_scaled, y_train)
 
selector.get_feature_names_out()

In [ ]:
# оцените, какие признаки демонстрируют наибольшую  важность в модели градиентного бустинга
fig, ax = plt.subplots(figsize=(13, 5)) #фигура + координатная плоскость
feature = X_train.columns #признаки
feature_importances = gb.feature_importances_ #важность признаков
#Строим столбчатую диаграмму
sns.barplot(x=feature, y=feature_importances, ax=ax);
#Добавляем подпись графику, осям абсцисс и ординат
ax.set_title('Bar plot feature importances')
ax.set_xlabel('Features')
ax.set_ylabel('Importances')
plt.xticks(rotation = 90);

In [ ]:
# реализуем оптимизацию гиперпараметров с помощью Optuna
def optuna_rf(trial):
# задаем пространства поиска гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 70, 200, 1)
    max_depth = trial.suggest_int('max_depth', 5, 30, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10, 1)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 6, 1)
    feat = ['sqrt', 'log2']
    max_features = trial.suggest_categorical('max_features', feat)
    crit = ['log_loss', 'gini', 'entropy']
    criterion = trial.suggest_categorical('criterion', crit)

  # создаем модель
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                        max_depth=max_depth,
                                        min_samples_leaf=min_samples_leaf,
                                        min_samples_split=min_samples_split,
                                        max_features=max_features,
                                        criterion=criterion,
                                        random_state=42)
  # обучаем модель
    model.fit(X_train_scaled, y_train)
    score_rf = metrics.f1_score(y_train, model.predict(X_train_scaled))
  
  # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score_rf = cross_val_score(model, X, y, cv=10, scoring="f1", n_jobs=-1).mean()
  

    return score_rf

In [ ]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_rf = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_rf.optimize(optuna_rf, n_trials=20)

In [ ]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_rf.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_rf.best_value))

In [ ]:
# рассчитаем точность для тестовой выборки
model_rf = ensemble.RandomForestClassifier(**study_rf.best_params,random_state=42, )
model_rf.fit(X_train_scaled, y_train)
y_train_pred = model_rf.predict(X_train_scaled)
print(metrics.classification_report(y_train, y_train_pred))
y_test_pred = model_rf.predict(X_test_scaled)
print(metrics.classification_report(y_test, y_test_pred))

In [ ]:
#Нас интересует только вероятность класса (второй столбец)
y_test_proba_pred_rf = model_rf.predict_proba(X_test_scaled)[:, 1]
#Для удобства завернем numpy-массив в pandas Series
y_test_proba_pred_rf = pd.Series(y_test_proba_pred_rf)
#Создадим списки, в которых будем хранить значения метрик 
recall_scores = []
precision_scores = []
f1_scores = []
#Сгенерируем набор вероятностных порогов в диапазоне от 0.1 до 1
thresholds = np.arange(0.1, 1, 0.05)
#В цикле будем перебирать сгенерированные пороги
for threshold in thresholds:
    #В противном случае - к классу 0
    y_test_pred = y_test_proba_pred_rf.apply(lambda x: 1 if x>threshold else 0)
    #Считаем метрики и добавляем их в списки
    recall_scores.append(metrics.recall_score(y_test, y_test_pred))
    precision_scores.append(metrics.precision_score(y_test, y_test_pred))
    f1_scores.append(metrics.f1_score(y_test, y_test_pred))

#Визуализируем метрики при различных threshold
fig, ax = plt.subplots(figsize=(10, 4)) #фигура + координатная плоскость
#Строим линейный график зависимости recall от threshold
ax.plot(thresholds, recall_scores, label='Recall')
#Строим линейный график зависимости precision от threshold
ax.plot(thresholds, precision_scores, label='Precision')

#Строим линейный график зависимости F1 от threshold
ax.plot(thresholds, f1_scores, label='F1-score')
#Даем графику название и подписи осям
ax.set_title('Recall/Precision dependence on the threshold')
ax.set_xlabel('Probability threshold')
ax.set_ylabel('Score')
ax.legend();

In [ ]:

#Задаем оптимальный порог вероятностей
threshold_opt = 0.39
y_test_pred_opt_rf = y_test_proba_pred_rf.apply(lambda x: 1 if x > threshold_opt else 0)
#Считаем метрики

print(metrics.classification_report(y_test, y_test_pred_opt_rf))

In [ ]:
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
    pen = ['l2', None]
    sol = ['lbfgs', 'saga', 'sag']
    penalty = trial.suggest_categorical('penalty', pen)
    solver = trial.suggest_categorical('solver', sol)
    max_iter = trial.suggest_int('max_iter', 100, 10000, 100)
    C = trial.suggest_uniform('C', 0.01, 1)

  # создаем модель
    model_logreg = LogisticRegression(penalty=penalty,
                                solver=solver,
                                max_iter=max_iter,
                                C=C,
                                random_state=42)
  # обучаем модель
    model_logreg.fit(X_train, y_train)
    score_logreg = metrics.f1_score(y_train, model_logreg.predict(X_train))
  
    return score_logreg

In [ ]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_logreg = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_logreg.optimize(optuna_lr, n_trials=20)

In [ ]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_logreg.best_params))
print("F1 на обучающем наборе: {:.2f}".format(study_logreg.best_value))

In [ ]:
# рассчитаем точность для тестовой выборки
model_logreg = LogisticRegression(**study_logreg.best_params,random_state=42, )
model_logreg.fit(X_train_scaled, y_train)
y_train_pred = model_logreg.predict(X_train_scaled)
print(metrics.classification_report(y_train, y_train_pred))
y_test_pred = model_logreg.predict(X_test_scaled)
print(metrics.classification_report(y_test, y_test_pred))

In [ ]:
#Нас интересует только вероятность класса (второй столбец)
y_test_proba_pred_logreg = model_logreg.predict_proba(X_test_scaled)[:, 1]
#Для удобства завернем numpy-массив в pandas Series
y_test_proba_pred_logreg = pd.Series(y_test_proba_pred_logreg)
#Создадим списки, в которых будем хранить значения метрик 
recall_scores = []
precision_scores = []
f1_scores = []
#Сгенерируем набор вероятностных порогов в диапазоне от 0.1 до 1
thresholds = np.arange(0.1, 1, 0.05)
#В цикле будем перебирать сгенерированные пороги
for threshold in thresholds:
    #В противном случае - к классу 0
    y_test_pred = y_test_proba_pred_logreg.apply(lambda x: 1 if x>threshold else 0)
    #Считаем метрики и добавляем их в списки
    recall_scores.append(metrics.recall_score(y_test, y_test_pred))
    precision_scores.append(metrics.precision_score(y_test, y_test_pred))
    f1_scores.append(metrics.f1_score(y_test, y_test_pred))

#Визуализируем метрики при различных threshold
fig, ax = plt.subplots(figsize=(10, 4)) #фигура + координатная плоскость
#Строим линейный график зависимости recall от threshold
ax.plot(thresholds, recall_scores, label='Recall')
#Строим линейный график зависимости precision от threshold
ax.plot(thresholds, precision_scores, label='Precision')

#Строим линейный график зависимости F1 от threshold
ax.plot(thresholds, f1_scores, label='F1-score')
#Даем графику название и подписи осям
ax.set_title('Recall/Precision dependence on the threshold')
ax.set_xlabel('Probability threshold')
ax.set_ylabel('Score')
ax.legend();

In [ ]:

#Задаем оптимальный порог вероятностей
threshold_opt = 0.5
y_test_pred_opt_logreg = y_test_proba_pred_logreg.apply(lambda x: 1 if x > threshold_opt else 0)
#Считаем метрики

print(metrics.classification_report(y_test, y_test_pred_opt_logreg))

In [ ]:
test_data_scaled

In [ ]:
new_test_data

In [ ]:
# рассчитаем точность для тестовой выборки
pred_gb = gb.predict(test_data_scaled)


In [ ]:
pass_ID = new_test_data['PassengerId']

In [ ]:
pass_ID

In [ ]:
output = pd.DataFrame({'PassengerId': pass_ID, 'Survived': pred_gb})

In [ ]:
output

In [ ]:
output.to_csv('GB_submis_less_features_z_score.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
output